# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [11]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [12]:
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


AUTH_DATA = os.environ['GIGACHAT_API_KEY']

llm = GigaChat(temperature=0.5, verify_ssl_certs=False, credentials=AUTH_DATA, scope='GIGACHAT_API_PERS')

Создадаим объект *memory* класса ***ConversationBufferMemory***

Класс ***ConversationBufferMemory*** отвечает за хранение диалогов с языковой моделью.

Соответственно объекты класса ***ConversationChain*** отвечает за интерфейс ведения диалога с языковой моделью. При создании объекта в качестве параметров передаётся ранее объявленная модель и буфер диалога.

In [18]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [19]:
conversation.predict(input="Привет! Меня зовут Максим!")



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Привет! Меня зовут Максим!
AI:

> Finished chain.


'Привет, Максим! Я рад с вами познакомиться. Как я могу вам помочь?'

In [20]:
conversation.predict(input="Сколько будет 2+3?")



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Привет! Меня зовут Максим!
AI: Привет, Максим! Я рад с вами познакомиться. Как я могу вам помочь?
Human: Сколько будет 2+3?
AI:

> Finished chain.


'5'

In [21]:
conversation.predict(input="Как меня зовут?")



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Привет! Меня зовут Максим!
AI: Привет, Максим! Я рад с вами познакомиться. Как я могу вам помочь?
Human: Сколько будет 2+3?
AI: 5
Human: Как меня зовут?
AI:

> Finished chain.


'Максим'

Как мы видим, назвав в первом сообщении своё имя, а в третьем - спросив его, модель верно ответила на этот вопрос. Соответственно, это подтверждение того, что с ней ввёлся диалог и сообщения не были изолированы друг от друга.

Следующей командой можно вывести весь буфер памяти диалога

In [22]:
print(memory.buffer)

Human: Привет! Меня зовут Максим!
AI: Привет, Максим! Я рад с вами познакомиться. Как я могу вам помочь?
Human: Сколько будет 2+3?
AI: 5
Human: Как меня зовут?
AI: Максим


In [23]:
memory.load_memory_variables({})

{'history': 'Human: Привет! Меня зовут Максим!\nAI: Привет, Максим! Я рад с вами познакомиться. Как я могу вам помочь?\nHuman: Сколько будет 2+3?\nAI: 5\nHuman: Как меня зовут?\nAI: Максим'}

На самом деле, каждый запрос к модели - независим.

In [24]:
memory = ConversationBufferMemory()

In [25]:
memory.save_context({"input": "Привет!"}, 
                    {"output": "Как ты?"})

In [26]:
print(memory.buffer)

Human: Привет!
AI: Как ты?


In [27]:
memory.load_memory_variables({})

{'history': 'Human: Привет!\nAI: Как ты?'}

In [28]:
memory.save_context({"input": "Да потихоньку."}, 
                    {"output": "Кул!"})

In [29]:
memory.load_memory_variables({})

{'history': 'Human: Привет!\nAI: Как ты?\nHuman: Да потихоньку.\nAI: Кул!'}

In [33]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

conversation.predict(input='Ну и чего?')



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Привет!
AI: Как ты?
Human: Да потихоньку.
AI: Кул!
Human: Ну и чего?
AI: Я просто хотел узнать, как у тебя дела.
Human: Ну и чего?
AI: Я просто хотел узнать, как у тебя дела.
Human: Ну и чего?
AI:

> Finished chain.


'Я просто хотел узнать, как у тебя дела.'

Как видим, мы смогли вручную смоделировать диалог с моделью, а затем на основе этого уже продолжить общаться в контексте вручную смоделированного диалога.

Соответственно можно сделать вывод, что любое сообщение в "диалоге" для модели - это очередной изолированный запрос с передачей в неё промпта со всем предыдущим диалогом + новым последним сообщением от пользователя.

И чем длиннее диалог, тем больше токенов тратится на каждый новый запрос. 

Помимо этого, с увеличением объема диалога, модель может начать "забывать" то, о чём говорилось в начале, так как израсходуется лимит её контекстного окна (хотя обычно за это не стоит переживать, так как у нынешних моделей оно достаточно большое)

## ConversationBufferWindowMemory

Далее рассмотрим класс ***ConversationBufferWindowMemory***, которой является тем же буфером диалога с одной особенностью - мы задаём размер скользящего окна запоминаемого диалога.

In [35]:
from langchain.memory import ConversationBufferWindowMemory

In [36]:
# k = 1 => запоминается одно сообщение от Human и AI
memory = ConversationBufferWindowMemory(k=1)               

In [37]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [38]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [39]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [40]:
conversation.predict(input="Привет, я Максим!")

AUTHENTICATION ERROR


'Привет, Максим! Я рад с вами познакомиться. Как прошёл ваш день?'

In [41]:
conversation.predict(input="Сколько будет 1+3?")

'4'

In [42]:
conversation.predict(input="Как меня зовут?")

'Ваше имя не было предоставлено мне для обработки.'

In [43]:
print(memory.buffer)

Human: Как меня зовут?
AI: Ваше имя не было предоставлено мне для обработки.


## ConversationTokenBufferMemory

Следующая разновидность буфера ограничивает объем запоминаемого диалога по количеству токенов. Это удобно в том смысле, что можно предсказать максимальную цену одного запроса к модели (т.к. стоимость обращения к модели через API всегда рассчитывается на основе количества токенов запроса). Все самые старые сообщения, которые не вошли в лимит токенов - удаляются.

In [44]:
from langchain.memory import ConversationTokenBufferMemory

In [47]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=25)
memory.save_context({"input": "Что такое ИИ"},
                    {"output": "Безумия!"})
memory.save_context({"input": "Обратное распространение ошибки, что это?"},
                    {"output": "Красота!"})
memory.save_context({"input": "Кто такие чатботы?"}, 
                    {"output": "Умельцы!"})

In [48]:
memory.load_memory_variables({})

{'history': 'AI: Безумия!\nHuman: Обратное распространение ошибки, что это?\nAI: Красота!\nHuman: Кто такие чатботы?\nAI: Умельцы!'}

Как можно увидеть, в лимит 25 токенов не уместилось первое сообщение от Human.

## ConversationSummaryMemory

Последний класс - самый интересный!

***ConversationSummaryMemory*** - это класс, реализующий механизм ограничения памяти по токенам и при превышении лимита делающий суммаризацию (пересказ) всего предыдущего диалога, чтобы уменьшить количество используемых токенов, но не терять предыдущую информацию.

In [49]:
from langchain.memory import ConversationSummaryBufferMemory

In [50]:
# create a long string
schedule = "Сегодня в 10 вечера собираемся на Думской на день рождение Крис!\
            Там будут все наши бывшие одноклассники - Митя, Ден, Саня и многие другие.\
            Ты же наверняка давно с ними не виделся, постоянно в работе. Приходи, хоть отдохнёшь \
            от своей заунывной работы. Кстати, не забудь про подарок. Крис всё же девочка требовательная \
            так что постарайся не поскупиться. Благо ты у нас богач! Давай, на цифрах."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=400)
memory.save_context({"input": "Привет!"}, {"output": "Как ты?"})
memory.save_context({"input": "Да ничего так, откисаю."},
                    {"output": "Круто!"})
memory.save_context({"input": "Какие планы на сегодня?"}, 
                    {"output": f"{schedule}"})

In [51]:
memory.load_memory_variables({})

{'history': 'Human: Привет!\nAI: Как ты?\nHuman: Да ничего так, откисаю.\nAI: Круто!\nHuman: Какие планы на сегодня?\nAI: Сегодня в 10 вечера собираемся на Думской на день рождение Крис!            Там будут все наши бывшие одноклассники - Митя, Ден, Саня и многие другие.            Ты же наверняка давно с ними не виделся, постоянно в работе. Приходи, хоть отдохнёшь             от своей заунывной работы. Кстати, не забудь про подарок. Крис всё же девочка требовательная             так что постарайся не поскупиться. Благо ты у нас богач! Давай, на цифрах.'}

Пока всё осталось так, как есть. Попробуем ограничить количество токенов.

In [54]:
# create a long string
schedule = "Сегодня в 10 вечера собираемся на Думской на день рождение Крис!\
            Там будут все наши бывшие одноклассники - Митя, Ден, Саня и многие другие.\
            Ты же наверняка давно с ними не виделся, постоянно в работе. Приходи, хоть отдохнёшь \
            от своей заунывной работы. Кстати, не забудь про подарок. Крис всё же девочка требовательная \
            так что постарайся не поскупиться. Благо ты у нас богач! Давай, на цифрах."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=70)
memory.save_context({"input": "Привет!"}, {"output": "Как ты?"})
memory.save_context({"input": "Да ничего так, откисаю."},
                    {"output": "Круто!"})
memory.save_context({"input": "Какие планы на сегодня?"}, 
                    {"output": f"{schedule}"})

In [55]:
memory.load_memory_variables({})

{'history': 'System: Человеку предлагают присоединиться к встрече одноклассников на дне рождения Крис на Думской в 10 вечера. Он должен принести подарок, так как Крис требовательна.'}

Здесь уже непосредственно можно увидеть реализацию механизма суммаризации всего предыдущего диалога.

Попробуем продолжить диалог.

In [58]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [59]:
conversation.predict(input="Где ещё раз встречаемся?")



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
System: Человеку предлагают присоединиться к встрече одноклассников на дне рождения Крис на Думской в 10 вечера. Он должен принести подарок, так как Крис требовательна.
Human: Кто-кто я?)
AI: Вы, человек.
Human: Где ещё раз встречаемся?
AI:

> Finished chain.


'На дне рождении Крис на Думской в 10 вечера.'

Данный механизм позволяет эффективнее вести диалог, тратя меньше токенов и не теряя важную информацию по ходу разговора.

# Альтернативные виды памяти

## Векторная база данных

Хранит текст в векторной базе данных и по запросу возвращает наиболее релевантные блоки текста.

## Воспоминания сущностей

Использую языковую модель, она запоминает детали об определенных сущностях.


Вы также можете использовать несколько видов памяти в один момента. У примеру, буфер диалога + воспоминания о сущностях, чтобы вести разговор об одном и том же с несколькими людьми.

Ещё можно хранить диалог в обычной базе данных, как например БД ключ-значение или реляционная БД.